# Classification using Neural Network

In [ ]:
import torch
import numpy as np
import pandas as pd

## Dataset

In [ ]:
import torchvision.datasets as dataset
import torchvision.transforms as transforms
train_data = dataset.MNIST(root="mnist", train=True, transform=transforms.ToTensor(), download=True)
test_data = dataset.MNIST(root="mnist", train=False, transform=transforms.ToTensor(), download=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 208344205.52it/s]


Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 104879388.59it/s]


Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 164226144.49it/s]

Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 2135470.10it/s]

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



In [ ]:
import torch.utils.data as data_utils
# batch_size indicates how many data is retrieved each time for training
# this allows to not process all data at one time as it may overload the CPU or GPU
train_loader = data_utils.DataLoader(dataset=train_data, batch_size=64, shuffle=True) # help to load a number of batch_size data
test_loader = data_utils.DataLoader(dataset=test_data, batch_size=64, shuffle=True)

## Network

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv = torch.nn.Sequential(
        # 1 indicates the channel, 32 indicates the number of kernel
        torch.nn.Conv2d(1, 32, kernel_size=5, padding=2),
        torch.nn.BatchNorm2d(32),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(2)
    )

    self.fc = torch.nn.Linear(14 * 14 * 32, 10)

  def forward(self, x):
    out = self.conv(x)
    out = out.view(out.size()[0], -1)
    out = self.fc(out)
    return out

cnn = CNN()
cnn = cnn.cuda()

## Loss

In [ ]:
loss_func = torch.nn.CrossEntropyLoss()

## Optimizer

In [ ]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.01)

## Training

In [ ]:
for epoch in range(10):
  for i, (image, labels) in enumerate(train_loader):
    images = image.cuda()
    labels = labels.cuda()

    outputs = cnn(images)
    loss = loss_func(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print("epoch is {}, iter is {}/{}, loss is {}".format(epoch + 1, i, len(train_data) // 64, loss))

流式输出内容被截断，只能显示最后 5000 行内容。
epoch is 5, iter is 628/937, loss is 0.014514862559735775
epoch is 5, iter is 629/937, loss is 0.08954882621765137
epoch is 5, iter is 630/937, loss is 0.03903740271925926
epoch is 5, iter is 631/937, loss is 0.005590129178017378
epoch is 5, iter is 632/937, loss is 0.05216412618756294
epoch is 5, iter is 633/937, loss is 0.05058770999312401
epoch is 5, iter is 634/937, loss is 0.11316950619220734
epoch is 5, iter is 635/937, loss is 0.004264278803020716
epoch is 5, iter is 636/937, loss is 0.03705015406012535
epoch is 5, iter is 637/937, loss is 0.008901325985789299
epoch is 5, iter is 638/937, loss is 0.06234424188733101
epoch is 5, iter is 639/937, loss is 0.024783222004771233
epoch is 5, iter is 640/937, loss is 0.03440411761403084
epoch is 5, iter is 641/937, loss is 0.05059809610247612
epoch is 5, iter is 642/937, loss is 0.024063486605882645
epoch is 5, iter is 643/937, loss is 0.040092870593070984
epoch is 5, iter is 644/937, loss is 0.034336939454078

## Testing

In [ ]:
loss_test = 0
accuracy = 0
for i, (images, labels) in enumerate(test_loader):
  images = images.cuda()
  labels = labels.cuda()

  outputs = cnn(images)
  loss_test += loss_func(outputs, labels)

  # Note: labels has a size of [bacthsize]
  # whereas outputs has a size of batchsize * cls (number of classes)
  # Note: each cls has a probability
  _, pred = outputs.max(1) # obtain a collection of index with max value on dimension 1
  accuracy += (pred == labels).sum().item()

accuracy = accuracy / len(test_data)
loss = loss_test / (len(test_data) // 64)

print(loss)
print(accuracy)

tensor(0.0611, device='cuda:0', grad_fn=<DivBackward0>)
0.9859


## Save Model

In [ ]:
torch.save(cnn, "mnist_model.pkl")

## Load Model

In [ ]:
torch.load("mnist_model.pkl")

CNN(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)